# Introduction
Here, we'll explore neural networks applied to the fashion-mnist problem.

In [140]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import csv
import pandas as pd

In [141]:
# Functions

def formatArray (dataFrame, columnToExtract) :
    array = dataFrame.values
    target = array[:,columnToExtract]
    params = np.delete(array, columnToExtract, axis = 1)
    return params, target

def loadFashionTrainData():
    return pd.read_csv("fashion-mnist-dataset/fashion-mnist_train.csv")

def loadFashionTestData():
    return pd.read_csv("fashion-mnist-dataset/fashion-mnist_test.csv")

def split_train_test(data, test_ratio):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

def createTarget (target):
    results = np.zeros((target.size, 10), dtype=int)
    for i in range(10):
        for j in range(target.size):
            if (target[j] != i):
                results[j][i - 1] = 0
            else:
                results[j][i - 1] = 1
    return results

# The dataset
First and foremost, we'll open train and test data. The training data is split to obtain validation items and the the target values are also separated from the original data.

In [142]:
fashionTrainDataset = loadFashionTrainData()
fashionTestDataset = loadFashionTestData()
fashionTrain, fashionValidation = split_train_test(fashionTrainDataset, 0.2)
fashionTrainParams, fashionTrainTarget = formatArray(fashionTrain, 0)
fashionValidationParams, fashionValidationTarget = formatArray(fashionValidation, 0)
print (fashionTrainParams[:5])
print (type(fashionTrainParams))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'numpy.ndarray'>


# Activation and Softmax Functions
First, we'll start by implementing some useful functions seen in class

In [143]:
def sigmoid(n):
    return (1/(1+np.exp(-n)))

def derivative_sigmoid(n):
    x = sigmoid(n)
    return x * (1 - x)

In [144]:
def relu(n):
    if n < 0:
        return 0
    return n

def derivative_relu(n):
    if n < 0:
        return 0
    return 1

In [145]:
def leaky_relu(n):
    if n > 0:
        return n
    return 0.01 * n

def derivative_leaky_relu(n):
    if n < 0:
        return 0.01
    return 1

In [160]:
def softmax(n):
    aux = np.exp(n)
    return aux/ np.sum(aux, axis=0)

## Forward Propagation
In this section, we define forward propagation related functions.

In [149]:
def initialize_1hl(input_dimension,hidden_layer_1_neurons, output_dimension):
    neural_data = {}
    np.random.seed(0)
    neural_data['w1'] = np.random.randn(input_dimension, hidden_layer_1_neurons)/ np.sqrt(input_dimension)
    neural_data['w2'] = np.random.randn(hidden_layer_1_neurons, output_dimension)/ np.sqrt(hidden_layer_1_neurons)
    
    neural_data['b1'] = np.zeros((1, hidden_layer_1_neurons))
    neural_data['b2'] = np.zeros((1, output_dimension))
    return neural_data

def initialize_2hl(input_dimension, hidden_layer_1_neurons, hidden_layer_2_neurons, output_dimension):
    neural_data = {}
    np.random.seed(0)

    neural_data['w1'] = np.random.randn(input_dimension, hidden_layer_1_neurons)/ np.sqrt(input_dimension)
    neural_data['w2'] = np.random.randn(hidden_layer_1_neurons, hidden_layer_2_neurons)/ np.sqrt(hidden_layer_1_neurons)
    neural_data['w3'] = np.random.randn(hidden_layer_2_neurons, output_dimension)/ np.sqrt(hidden_layer_2_neurons)

    neural_data['b1'] = np.zeros((1, hidden_layer_1_neurons))
    neural_data['b2'] = np.zeros((1, hidden_layer_2_neurons))
    neural_data['b3'] = np.zeros((1, output_dimension))
    return neural_data
    
def forward_prop_1hl(x, neural_data):
    w1 , w2, b1, b2 = neural_data['w1'], neural_data['w2'], neural_data['b1'], neural_data['b2']
    x1 = np.dot(x, w1) + b1 #Output of hidden layer
    y1 = np.asarray([[relu(n) for n in j] for j in x1]) #Output of hidden layer with activation function
    x2 = np.dot(y1, w2) + b2 #Output of last layer
    neural_data['x1'] = x1
    neural_data['x2'] = x2
    neural_data['y1'] = y1
    neural_data['o'] = softmax(x2)
    return neural_data['o']

def forward_prop_2hl(x, neural_data):
    w1 , w2, w3, b1, b2, b3 = neural_data['w1'], neural_data['w2'], neural_data['w3'], neural_data['b1'], neural_data['b2'], neural_data['b3']

    x1 = np.dot(x, w1) + b1
    y1 = np.asarray([[relu(n) for n in j] for j in x1])
    x2 = np.dot(a1, w2) + b2
    y2 = np.asarray([[relu(n) for n in j] for j in x2])
    x3 = np.dot(a2, w3) + b3
    
    neural_data['x1'] = x1
    neural_data['x2'] = x2
    neural_data['x3'] = x3

    neural_data['y1'] = y1
    neural_data['y2'] = y2
    
    neural_data['o'] = softmax(x3)
    return neural_data['o']

## Prediction Functions
Helper functions that return predictions, given our model.

In [150]:
def predict_1hl(x, neural_data):
    print(x)
    print("=========================")
    test = forward_prop_1hl(x,neural_data)
    print(test)
    return np.argmax(test)

def predict_2hl(x, neural_data):
    return np.argmax(forward_prop_2hl(x,neural_data))

## Cost function

In [151]:
def gradient_cost(fashionTrainOutput, fashionTarget, fashionTargetMinusOne, testCasesAmount):
    final_cost = 0
    for j in testCasesAmount:
        cost = np.add(np.dot(fashionTarget, np.log10(fashionTrainOutput[j])),np.dot(fashionTargetMinusOne, (np.ones(10) - np.log10(fashionTrainOutput[j]))))
    final_cost = np.sum(cost)/testCasesAmount
    return final_cost

def neuralNetworkCostFunction(fashionTrainOutput, fashionTarget):
    diference = fashionTrainOutput - fashionTarget
    squareDiference = diference ** 2
    print(fashionTrainOutput)
    n = fashionTrainOutput.shape[0]    
    return (np.sum(squareDiference)/(2*n))

def regressionLogisticCostFunction (results, model, X):
    agaTheta = model.predict_proba(X)
    n = X.shape[0]
    diference = results - agaTheta
    squareDiference = diference * diference
    return (np.sum(squareDiference)/(2*n))

## Training and backward propagation

Some notes:

### For one hidden layer:
- x1 = #Output of hidden layer
- x2 = #Output of last layer
- y1 = #Output of hidden layer with activation function
- o = Final output with Softmax

### For TWO hidden layers:
- x1 = #Output of first hidden layer
- x2 = #Output of second hidden layer
- x3 = Output of last layer
- y1 = #Output of first hidden layer with activation function
- y2 = #Output of second hidden layer with activation function
- o = Final output with Softmax


# One hidden layer

Here, we present our code and results achieved by a learning algorithm that uses a neural network with only one hidden layer.

In [154]:
def train_neural_network_1hl(hidden_layer_1_neurons, epochs, trainParams, trainTarget, learning_rate, regularization_rate):
    input_dimension = 784
    output_dimension = 10
#     Initializes weights and biases for our neural network
    neural_data = initialize_1hl(input_dimension, hidden_layer_1_neurons, output_dimension)
    sigmoid = True
    relu = False
    leaky_relu = False
    
#     Prepares for mini-batch
    batchSize = 64
    start_idx = 0;
    indices = np.arange(trainParams.shape[0])
    np.random.shuffle(indices)
    
#     Performs Backpropagation
    for j in range(epochs):
        excerpt = indices[start_idx:start_idx + batchSize]
        mini_batch_data = trainParams[excerpt]
        miniBatchTarget = createTarget(trainTarget[excerpt])
#         Performs Forward propagation
        probs = forward_prop_1hl(mini_batch_data, neural_data)
    
#         Calculates cost
#         cur_result = np.amax(probs, axis=1)
        print("============================")
        print(probs)
#         print("============================")
#         neuralNetworkCostFunction(fashionTrainOutput, fashionTarget)

        print("============================")
#         Performs Backward propagation
        delta3 = probs - miniBatchTarget
        dW2 = (neural_data['y1'].T).dot(delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(neural_data['w2'].T)
        aux = neural_data['y1']
        if sigmoid: 
            for i in aux:
                i = derivative_sigmoid(i)
        if relu:
            for i in aux:
                i = derivative_relu(i)
        if leaky_relu:
            for i in aux:
                i = derivative_leaky_relu(i)        
        delta2 = delta2 * aux
        dW1 = np.dot(mini_batch_data.T, delta2)
        db1 = np.sum(delta2, axis=0)
        
        
         # Performs regularization
        dW2 += regularization_rate * neural_data['w2']
        dW1 += regularization_rate * neural_data['w1']

        # Gradient descent parameter update
        neural_data['w1'] += -learning_rate * dW1
        neural_data['b1'] += -learning_rate * db1
        neural_data['w2'] += -learning_rate * dW2
        neural_data['b2'] += -learning_rate * db2
        
        print("Ended iteration", j)
        start_idx += 1;
        start_idx %= mini_batch_data.shape[0]
        
    return neural_data

# Two hidden layers

Same as before, but for 2 hidden layers.

In [155]:
def train_neural_network_2hl(hidden_layer_1_neurons, hidden_layer_2_neurons, epochs, trainParams, trainTarget, learning_rate, regularization_rate):
    input_dimension = 784
    output_dimension = 10
#     Initializes weights and biases for our neural network
    neural_data = initialize_2hl(input_dimension, hidden_layer_1_neurons, hidden_layer_2_neurons, output_dimension)
    sigmoid = True
    relu = False
    leaky_relu = False
    
#     Prepares for mini-batch
    batchSize = 64
    start_idx = 0;
    indices = np.arange(trainParams.shape[0])
    np.random.shuffle(indices)
    excerpt = indices[start_idx:start_idx + batchSize]
    mini_batch_data = trainParams[excerpt]
    miniBatchTarget = createTarget(trainTarget[excerpt])
#     Performs Backpropagation
    for j in range(epochs):

#         Performs Forward propagation
        probs = forward_prop_1hl(mini_batch_data, neural_data)
        
#         Calculates cost

#         Performs Backward propagation
        delta4 = probs - miniBatchTarget
        dW3 = (neural_data['y2'].T).dot(delta4)
        db3 = np.sum(delta4, axis=0, keepdims=True)
        delta3 = delta4.dot(neural_data['w3'].T)
        aux = neural_data['y2']
        if sigmoid: 
            for i in aux:
                i = derivative_sigmoid(i)
        if relu:
            for i in aux:
                i = derivative_relu(i)
        if leaky_relu:
            for i in aux:
                i = derivative_leaky_relu(i)        
        delta3 = delta3 * aux
        dW2 = np.dot(mini_batch_data.T, delta3)
        db2 = np.sum(delta3, axis=0)
        delta2 = delta3.dot(neural_data['w2'].T)
        aux = neural_data['y1']
        if sigmoid: 
            for i in aux:
                i = derivative_sigmoid(i)
        if relu:
            for i in aux:
                i = derivative_relu(i)
        if leaky_relu:
            for i in aux:
                i = derivative_leaky_relu(i)
        delta2 = delta2 * aux
        dW1 = np.dot(mini_batch_data.T, delta2)
        db1 = np.sum(delta2, axis=0)      
        
         # Performs regularization
        dW3 += regularization_rate * neural_data['w3']
        dW2 += regularization_rate * neural_data['w2']
        dW1 += regularization_rate * neural_data['w1']

        # Gradient descent parameter update
        
        neural_data['w1'] += -learning_rate * dW1
        neural_data['b1'] += -learning_rate * db1
        neural_data['w2'] += -learning_rate * dW2
        neural_data['b2'] += -learning_rate * db2
        neural_data['w3'] += -learning_rate * dW3
        neural_data['b3'] += -learning_rate * db3
        
        print("Ended iteration", j)
        start_idx += 1;
        start_idx %= mini_batch_data.shape[0]
        
    return neural_data

# Testing the neural networks

Now, we'll test our neural networks under multiple circumstances on the validation set, so we can gest the best possible models.

## Problem details
- Input dimension: 28x28 = 784 neurons
- Output dimension: 10 classes = 10 neurons

In [156]:
hidden_layers = 1
input_neurons = 784
output_neurons = 10
hidden_layer_1_neurons = 15
hidden_layer_2_neurons = 15
learning_rate = 1
regularization_rate = 0
epochs = 10

In [157]:
# fashionTrainParams, fashionTrainTarget
model_1hl = train_neural_network_1hl(hidden_layer_1_neurons, epochs, fashionTrainParams, fashionTrainTarget, learning_rate, regularization_rate)


[[2.57343232e-036 1.56392700e-065 1.18408878e-020 5.82212266e-002
  2.78893282e-110 2.45296947e-049 6.17146223e-009 9.35762666e-066
  1.40691512e-025 5.15101425e-019]
 [2.70740034e-028 1.18641189e-064 3.27662951e-029 2.79767928e-043
  2.19180304e-031 4.47729613e-001 3.15304244e-037 1.00000000e+000
  7.61606007e-068 4.94270585e-025]
 [2.53476914e-016 2.70893568e-070 1.23773141e-025 1.02219295e-032
  2.47562631e-079 2.31449867e-054 4.99941421e-025 1.83695628e-038
  3.76945626e-049 1.55250007e-046]
 [1.69409212e-033 9.05345971e-058 4.36794146e-026 1.70036568e-024
  7.27292236e-106 1.12709768e-026 5.29790186e-014 5.20711590e-042
  1.18791706e-032 1.97214253e-033]
 [1.69201761e-041 1.49180308e-047 2.18110261e-013 1.48511785e-070
  3.45581598e-024 2.39610895e-027 1.43866367e-014 1.41040620e-043
  3.89150839e-034 1.15450119e-056]
 [2.59996769e-046 7.85173553e-025 8.23210206e-022 6.01156062e-020
  5.46182927e-073 6.65322138e-041 4.21520338e-039 1.02007054e-034
  1.83376699e-041 3.51610378e-019

/home/nacholez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [158]:
results = predict_1hl(fashionValidationParams, model_1hl)

[[  0   0   0 ...   0   0   0]
 [  0   0   0 ... 211 102   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [159]:
print(neuralNetworkCostFunction(results, fashionValidationTarget))

0


IndexError: tuple index out of range